In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-09-30T00:00:00Z,01001,"23.06.2021, 00:00 Uhr",0,-9,2020-09-30T00:00:00Z,0,1,0,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-10-29T00:00:00Z,01001,"23.06.2021, 00:00 Uhr",0,-9,2020-10-29T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-03T00:00:00Z,01001,"23.06.2021, 00:00 Uhr",0,-9,2020-11-03T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-20T00:00:00Z,01001,"23.06.2021, 00:00 Uhr",0,-9,2020-11-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-23T00:00:00Z,01001,"23.06.2021, 00:00 Uhr",0,-9,2020-11-18T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
494,2021-06-18,999,2,518,473,8,52,172,351,335,75,14,0,8048.0,1149.714286,0.61,83166711,9.68,Deutschland,Nation
495,2021-06-19,741,0,375,355,11,44,126,247,248,59,17,0,7483.0,1069.000000,0.65,83166711,9.00,Deutschland,Nation
496,2021-06-20,266,0,129,136,1,22,36,101,91,13,3,0,7236.0,1033.714286,0.72,83166711,8.70,Deutschland,Nation
497,2021-06-21,545,1,295,237,13,24,101,180,186,44,9,1,6859.0,979.857143,0.77,83166711,8.25,Deutschland,Nation
498,2021-06-22,820,2,432,380,8,38,134,270,285,70,22,1,5985.0,855.000000,0.74,83166711,7.20,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
7527,Thüringen,2021-06-18,16,24,2133378,0,11,13,0,3,...,0,0,30.75,25.142857,1.108108,0.58,176.0,8.25,Thüringen,Bundesland
7528,Thüringen,2021-06-19,16,13,2133378,0,6,7,0,1,...,0,0,23.00,22.714286,0.948454,0.58,159.0,7.45,Thüringen,Bundesland
7529,Thüringen,2021-06-20,16,4,2133378,0,2,2,0,0,...,0,0,14.00,22.857143,0.523364,0.65,160.0,7.50,Thüringen,Bundesland
7530,Thüringen,2021-06-21,16,18,2133378,1,8,10,0,0,...,0,0,14.75,22.571429,0.495798,0.75,158.0,7.41,Thüringen,Bundesland
7531,Thüringen,2021-06-22,16,18,2133378,0,10,8,0,1,...,2,0,13.25,18.857143,0.430894,0.75,132.0,6.19,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
147695,Städteregion Aachen,2021-06-18,05334,Nordrhein-Westfalen,11,557026.0,0,4,7,0,...,0,0,13.00,11.714286,1.333333,0.81,82.0,14.72,Städteregion Aachen,Landkreis
147696,Städteregion Aachen,2021-06-19,05334,Nordrhein-Westfalen,5,557026.0,0,2,3,0,...,0,0,9.75,9.857143,0.812500,0.78,69.0,12.39,Städteregion Aachen,Landkreis
147697,Städteregion Aachen,2021-06-20,05334,Nordrhein-Westfalen,1,557026.0,0,1,0,0,...,0,0,5.25,8.857143,0.428571,0.74,62.0,11.13,Städteregion Aachen,Landkreis
147698,Städteregion Aachen,2021-06-21,05334,Nordrhein-Westfalen,5,557026.0,0,4,1,0,...,0,0,5.50,9.000000,0.488889,0.79,63.0,11.31,Städteregion Aachen,Landkreis
147699,Städteregion Aachen,2021-06-22,05334,Nordrhein-Westfalen,1,557026.0,0,1,0,0,...,0,0,3.00,6.571429,0.230769,0.56,46.0,8.26,Städteregion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
147695,2021-06-18,11,0,4,7,0,3,2,4,1,...,557026.0,14.72,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,13.00,1.333333,Städteregion Aachen,05334
147696,2021-06-19,5,0,2,3,0,0,1,2,0,...,557026.0,12.39,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,9.75,0.812500,Städteregion Aachen,05334
147697,2021-06-20,1,0,1,0,0,0,0,0,0,...,557026.0,11.13,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,5.25,0.428571,Städteregion Aachen,05334
147698,2021-06-21,5,0,4,1,0,0,0,3,1,...,557026.0,11.31,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,5.50,0.488889,Städteregion Aachen,05334
147699,2021-06-22,1,0,1,0,0,0,0,0,0,...,557026.0,8.26,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,3.00,0.230769,Städteregion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')